# Создание датасета

## Загрузка необходимых модулей

In [1]:
import os
import pandas as pd
from collections import Counter
from tqdm import tqdm, trange
import re
from nltk import sent_tokenize

In [2]:
def count_words(text):
    """Считает количество слов в предложении"""
    return len(re.findall("\w+", text))

def sample_split(text, sample_size=500):
    """Создает из текста список его частей длиной примерно sample_size"""
    samples = []
    cur_sample = ''
    for sent in sent_tokenize(text):
        cur_sample += ' ' + sent 
        if count_words(sent) + count_words(cur_sample) > sample_size:
            samples.append(cur_sample)
            cur_sample = '' 
            
    if cur_sample != '':
        samples.append(cur_sample)
    return samples 

sample_split('Привет, о чем говориите? Расскажите все до мелочей! Я акккредитованый ккритик всех, даже мне не известных, вещей. Я не слушал, но ты не прав - попробуй меня оспорить. Тебя ждет монолог до утра с кучей неинтересных историй. Я стану лучше когда-нибудь позже, наверно. А может и нет.', 20)

[' Привет, о чем говориите? Расскажите все до мелочей! Я акккредитованый ккритик всех, даже мне не известных, вещей.',
 ' Я не слушал, но ты не прав - попробуй меня оспорить. Тебя ждет монолог до утра с кучей неинтересных историй.',
 ' Я стану лучше когда-нибудь позже, наверно. А может и нет.']

## Основанный на русской классике

In [26]:
def get_files(path: str):
    """Загрузка списка текстовых файлов из директории path"""
    if path[-1] != '/':
        path += '/'
    return [path + filename for filename in os.listdir(path) if filename[-3:] == 'txt']

train_files = get_files('../data/russian_classics/train')
test_files = get_files('../data/russian_classics/test')

In [23]:
def get_author_files(files: list) -> dict:
    """Разделение файлов по автору, указанному в формате автор_название.txt"""
    res = {}
    for filename in files:
        author = filename.split('/')[-1].split('_')[0]
        if author in res:
            res[author].append(filename)
        else:
            res[author] = [filename]

    return res

author_files_train = get_author_files(train_files)
author_files_test = get_author_files(test_files)

In [27]:
def create_dataset(author_files: dict, samples=5, sample_len=600, max_authors=30):
    """Создание датасета, где для каждого из max_authors автора есть samples фрагментов текста объемом sample_len слов"""
    res = []
    authors = sorted(list(author_files.keys()))[:max_authors]
    for author_index in trange(len(authors)):
        author = authors[author_index]
        collected_texts = 0
        loop_round = 0
        finished_files = []
        while collected_texts < samples:
            for filename in author_files[author]:
                with open(filename) as f:
                    sample_texts = sample_split(f.read(), sample_len)
                    if loop_round >= len(sample_texts):
                        if filename not in finished_files:
                            finished_files.append(filename)
                        continue 
                    text = sample_texts[loop_round]
                    if len(text) > 0:
                        res.append([authors[author_index], author_index, text])
                        collected_texts += 1
                    if collected_texts >= samples:
                        break
            loop_round += 1
            if len(finished_files) == len(author_files[author]):
                break
        if author_index >= max_authors:
            break

    df = pd.DataFrame(res, columns=['Author Name', 'Author', 'Content'])
    return df

In [25]:
TRAIN_SAMPLES = 5
TRAIN_SAMPLES_LEN = 600
TEST_SAMPLES = 40
TEST_SAMPLES_LEN = 1000
AUTHORS_AMOUNT = 23

df_train = create_dataset(author_files_train, TRAIN_SAMPLES, TRAIN_SAMPLES_LEN, AUTHORS_AMOUNT)
df_test = create_dataset(author_files_test, TEST_SAMPLES, TEST_SAMPLES_LEN, AUTHORS_AMOUNT)

In [22]:
df_train.to_csv('../datasets/russian_classics/train_{}_{}_{}.csv'.format(AUTHORS_AMOUNT, TRAIN_SAMPLES, TRAIN_SAMPLES_LEN))
df_test.to_csv('../datasets/russian_classics/test_{}_{}_{}.csv'.format(AUTHORS_AMOUNT, TEST_SAMPLES, TEST_SAMPLES_LEN))

## Основанный на материалах с proza.ru

In [25]:
proza_df = pd.read_csv('../datasets/proza_ru/proza_full.csv')

In [28]:
from collections import Counter
authors = list(dict(Counter(proza_df['Author'])).keys())
i = 0
author_dict = {}
for author in authors:
    author_dict[author] = i
    i += 1

def author_code(author_name):
    return author_dict[author_name]

proza_df['Author'] = proza_df['Author'].apply(author_code)

In [63]:
import random
import numpy as np
msk = np.random.rand((len(proza_df))) < 0.8


In [64]:
proza_train = proza_df[msk]
proza_test = proza_df[~msk]

In [65]:
proza_train.to_csv('../datasets/proza_ru/proza_train.csv')
proza_test.to_csv('../datasets/proza_ru/proza_test.csv')